# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

In [1]:
import sympy as sp
from ham_to_sparse_matrix import *
from constants import *
import scipy.sparse.linalg

In [9]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(4*aLat) + aLat*(qnP1-qnM1)*V(qn)/(4*aLat) )
fermionNI=sp.expand( -(xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [10]:
bosonNI+bosonI

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [11]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} - \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) - \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [12]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [13]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + q_{n+1}*V(q_{n})/4 - q_{n-1}*V(q_{n})/4 + V(q_{n})*q_{n+1}/4 - V(q_{n})*q_{n-1}/4 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} - \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) - \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [14]:
ham=0

mass=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)

def potential(n):
    # m*q with m=1
    return mass*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)


ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
sp.expand(ham)

0.5 - \chi^{\dagger}_{0}*\chi_{0} - \chi^{\dagger}_{0}*\chi_{1}/2 + \chi^{\dagger}_{0}*\chi_{2}/2 - \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{1} - \chi^{\dagger}_{1}*\chi_{2}/2 + \chi^{\dagger}_{2}*\chi_{0}/2 - \chi^{\dagger}_{2}*\chi_{1}/2 - \chi^{\dagger}_{2}*\chi_{2} + p_{0}**2/2 + p_{1}**2/2 + p_{2}**2/2 - q_{0}*q_{1}/8 - q_{0}*q_{2}/8 + 3*q_{0}**2/4 - q_{1}*q_{0}/8 - q_{1}*q_{2}/8 + 3*q_{1}**2/4 - q_{2}*q_{0}/8 - q_{2}*q_{1}/8 + 3*q_{2}**2/4

In [16]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

1/2 - \chi^{\dagger}_{0}*\chi_{0} - \chi^{\dagger}_{0}*\chi_{1}/2 + \chi^{\dagger}_{0}*\chi_{2}/2 - \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{1} - \chi^{\dagger}_{1}*\chi_{2}/2 + \chi^{\dagger}_{2}*\chi_{0}/2 - \chi^{\dagger}_{2}*\chi_{1}/2 - \chi^{\dagger}_{2}*\chi_{2} - a^{\dagger}_{0}*a^{\dagger}_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 5*a^{\dagger}_{0}*a_{0}/8 - a^{\dagger}_{0}*a_{1}/16 - a^{\dagger}_{0}*a_{2}/16 + a^{\dagger}_{0}**2/8 - a^{\dagger}_{1}*a^{\dagger}_{0}/16 - a^{\dagger}_{1}*a^{\dagger}_{2}/16 - a^{\dagger}_{1}*a_{0}/16 + 5*a^{\dagger}_{1}*a_{1}/8 - a^{\dagger}_{1}*a_{2}/16 + a^{\dagger}_{1}**2/8 - a^{\dagger}_{2}*a^{\dagger}_{0}/16 - a^{\dagger}_{2}*a^{\dagger}_{1}/16 - a^{\dagger}_{2}*a_{0}/16 - a^{\dagger}_{2}*a_{1}/16 + 5*a^{\dagger}_{2}*a_{2}/8 + a^{\dagger}_{2}**2/8 + 5*a_{0}*a^{\dagger}_{0}/8 - a_{0}*a^{\dagger}_{1}/16 - a_{0}*a^{\dagger}_{2}/16 - a_{0}*a_{1}/16 - a_{0}*a_{2}/16 + a_{0}**2/8 - a_{1}*a^{\dagger}_{0}/16 + 5*a_{1}*a^{\dagger}

In [17]:
cutoff=6
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)

/home/chris/anaconda3/envs/QuantumComputing/lib/python3.9/site-packages/scipy/sparse/_sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


In [18]:
hamMat

<1728x1728 sparse matrix of type '<class 'numpy.complex64'>'
	with 25632 stored elements in Compressed Sparse Row format>

In [19]:
ens=scipy.sparse.linalg.eigs(hamMat,k=10,sigma=0)[0]

In [20]:
ens

array([0.13672549+3.9363273e-09j, 0.82286537-5.7748132e-08j,
       1.1349455 +7.4264133e-09j, 1.3228645 +6.9571717e-09j,
       1.4574281 +3.6721159e-09j, 1.5420889 +2.3216097e-08j,
       1.4574282 +4.9628056e-08j, 1.8210869 -1.0007612e-08j,
       1.8228647 -1.6949008e-08j, 2.137434  -8.2583407e-10j],
      dtype=complex64)

In [22]:
for cutoff in [2,3,4,5,6,7,8]:
    t=Timer('toHam')
    t.start()
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    t.stop()
    
    t=Timer('evals')
    t.start()
    ens=scipy.sparse.linalg.eigs(hamMat,k=10,sigma=0)[0]
    t.stop()
    
    print("Lambda = ",cutoff)
    print(ens)
    print()

toHam timer took: 0.5395 seconds
evals timer took: 0.0028 seconds
Lambda =  2
[-0.18614052-2.68141376e-09j -0.18614064-2.62380784e-09j
  0.49999946-1.22417205e-08j  0.3138589 -7.83651810e-09j
  0.3138592 -6.33685104e-09j  0.49999994+1.54552211e-08j
  0.31385913-6.51119203e-09j  0.31385934+5.41898570e-09j
  0.3138593 +9.67022373e-09j  0.313859  +5.79043169e-09j]

toHam timer took: 0.9285 seconds
evals timer took: 0.0069 seconds
Lambda =  3
[0.04649036-1.7305629e-10j 0.72558075+3.7998625e-08j
 0.73262906+1.5942334e-09j 0.7576407 -1.4564307e-08j
 0.757642  -6.9625941e-09j 1.1678125 -1.1196752e-08j
 1.2326286 -1.0638945e-09j 1.3138535 +3.5425032e-08j
 1.3475709 -1.8859371e-08j 1.3475657 +5.3891728e-08j]

toHam timer took: 2.1516 seconds
evals timer took: 0.0105 seconds
Lambda =  4
[0.13618696-6.4437855e-10j 0.8223259 +2.6281466e-09j
 0.9515979 +3.2562241e-08j 1.1648198 +1.1704689e-08j
 1.322325  -7.2710748e-08j 1.1648233 +4.2269864e-08j
 1.5415499 +9.2188417e-09j 1.5737243 +1.3878100e-08j
